# 1 准备阶段

## 1.1 导入必要包

In [34]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import torchvision

import os
import time

from PIL import Image

## 1.2 准备数据集

In [35]:
train_data = torchvision.datasets.CIFAR10(
    root="../data/CIFAR10/", 
    train=True, 
    transform=torchvision.transforms.ToTensor(),
    download=True
)

test_data = torchvision.datasets.CIFAR10(
    root="../data/CIFAR10/", 
    train=False, 
    transform=torchvision.transforms.ToTensor(),
    download=True
)

## 1.3 数据集预览和预处理

In [36]:
train_data_size = len(train_data)
test_data_size = len(test_data)
print("训练集的数据长度为：{}".format(train_data_size))
print("测试集的数据长度为：{}".format(test_data_size))

训练集的数据长度为：50000
测试集的数据长度为：10000


利用`DataLoader`加载数据

In [37]:
train_dataloader = DataLoader(dataset=train_data, batch_size=64)
test_dataloader = DataLoader(dataset=test_data, batch_size=64)

# 2 搭建神经网络

<p align="center">
  <img src="https://penry-1352739541.cos.ap-beijing.myqcloud.com/img/20251224124947699.png" alt="" width="55%" height="height">
</p>

In [38]:
class Model_CIFAR(nn.Module):
    def __init__(self):
        super(Model_CIFAR, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten()
        )

        self.classfier = nn.Sequential(
            nn.Linear(in_features=64*4*4, out_features=64),
            nn.Linear(in_features=64, out_features=10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classfier(x)
        return x

# 3 创建模型及后续步骤

只有以下类型可以用cuda加速，也就是放到GPU上：
- 网络模型
- 损失函数
- 数据

第一种：使用以下操作转移到GPU上操作
- `model.cuda()`
- `loss_fn.cuda()`
- `imgs.cuda()`
- `targets.cuda()`

In [39]:
# 创建网络模型
model = Model_CIFAR()
model = model.cuda()

# 创建损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.cuda()

# 定义优化器
learning_rate = 1e-2
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)

# 设置训练网络参数
epoch = 20 # 训练轮次
total_train_step = 0 # 记录训练次数
total_test_step = 0 # 记录测试次数
best_accuracy = 0.0 # 记录最佳准确率

# 添加tensorboard
writer = SummaryWriter('../logs/logs_train')

start_time = time.time()
for i in range(epoch):
    print(f"-------第{i+1}轮训练开始-------")

    # 训练开始
    model.train()
    total_train_loss = 0.0
    avg_train_loss = 0.0
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.cuda()
        targets = targets.cuda()
        output = model(imgs)
        loss = loss_fn(output, targets)
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1
        total_train_loss += loss.item()

        # 每100个batch打印一次
        if total_train_step % 100 == 0:
            end_time = time.time()
            print(f"训练100个batch耗时：{end_time - start_time}秒")
            print(f"训练次数：{total_train_step}， Loss：{loss.item()}")
            writer.add_scalar(tag='train_loss', scalar_value=loss.item(), global_step=total_train_step)

    # 打印每个epoch的平均loss
    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"第 {i+1} 轮训练，平均 Loss：{avg_train_loss:.4f}")

    # 测试阶段
    model.eval()
    total_test_loss= 0.0
    total_accuracy = 0.0
    avg_test_loss = 0.0

    with torch.no_grad(): # 测试时候不需要计算梯度
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.cuda()
            targets = targets.cuda()
            outputs = model(imgs)
            loss = loss_fn(outputs, targets)

            total_test_loss += loss.item()

            # 计算准确率
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy += accuracy

    avg_test_loss = total_test_loss / len(test_dataloader)
    overall_accuracy = total_accuracy / test_data_size
    print(f"第 {i+1} 轮测试，平均 Loss：{avg_test_loss:.4f}，准确率：{overall_accuracy:.2%}")

    writer.add_scalar(tag='test_loss', scalar_value=avg_test_loss, global_step=total_test_step)

    total_test_step += 1

    # 模型保存：每5个epoch保存一次模型
    if (i+1)%5 == 0:
        torch.save(model.state_dict(), f"../checkpoints/CIFRA10/model_epoch_{i+1}.pth")
        print(f"保存第{i+1}轮次的模型")

    # 最佳模型保存
    if overall_accuracy > best_accuracy:
        best_accuracy = overall_accuracy
        torch.save(model.state_dict(), "../checkpoints/CIFRA10/best_model.pth")
        print(f"保存最佳模型！准确率：{best_accuracy:.2%}")

    print()

writer.close()

print(f"🎉 训练完成！最佳准确率：{best_accuracy:.2%}")
print(f"模型已保存到 ../checkpoints/ 目录")

-------第1轮训练开始-------
训练100个batch耗时：0.34072208404541016秒
训练次数：100， Loss：2.286067485809326
训练100个batch耗时：0.6848113536834717秒
训练次数：200， Loss：2.294322967529297
训练100个batch耗时：1.0136561393737793秒
训练次数：300， Loss：2.259579658508301
训练100个batch耗时：1.3355181217193604秒
训练次数：400， Loss：2.1823830604553223
训练100个batch耗时：1.6576180458068848秒
训练次数：500， Loss：2.147213935852051
训练100个batch耗时：1.980560064315796秒
训练次数：600， Loss：2.116014242172241
训练100个batch耗时：2.2948033809661865秒
训练次数：700， Loss：1.9936423301696777
第 1 轮训练，平均 Loss：2.1812
第 1 轮测试，平均 Loss：1.9707，准确率：28.04%
保存最佳模型！准确率：28.04%

-------第2轮训练开始-------
训练100个batch耗时：2.995689630508423秒
训练次数：800， Loss：1.83610999584198
训练100个batch耗时：3.3105227947235107秒
训练次数：900， Loss：1.7453320026397705
训练100个batch耗时：3.6254866123199463秒
训练次数：1000， Loss：1.907186508178711
训练100个batch耗时：3.9419126510620117秒
训练次数：1100， Loss：2.043125629425049
训练100个batch耗时：4.2561328411102295秒
训练次数：1200， Loss：1.6853785514831543
训练100个batch耗时：4.593693017959595秒
训练次数：1300， Loss：1.5909383296966553
训练1

第二种：使用定义设备
- `device = torch.device("cpu")`
- `device = torch.device("cuda")`
- `model = model.to(device)`
- `loss_fn = loss_fn.to(device)`
- `imgs = imgs.to(device)`
- `targets = targets.to(device)`

In [40]:
# 定义训练的设备
device = torch.device("cpu")

# 创建网络模型
model = Model_CIFAR()
model.to(device)

# 创建损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device)

# 定义优化器
learning_rate = 1e-2
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)

# 设置训练网络参数
epoch = 20 # 训练轮次
total_train_step = 0 # 记录训练次数
total_test_step = 0 # 记录测试次数
best_accuracy = 0.0 # 记录最佳准确率

# 添加tensorboard
writer = SummaryWriter('../logs/logs_train')

start_time = time.time()
for i in range(epoch):
    print(f"-------第{i+1}轮训练开始-------")

    # 训练开始
    model.train()
    total_train_loss = 0.0
    avg_train_loss = 0.0
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        output = model(imgs)
        loss = loss_fn(output, targets)
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1
        total_train_loss += loss.item()

        # 每100个batch打印一次
        if total_train_step % 100 == 0:
            end_time = time.time()
            print(f"训练100个batch耗时：{end_time - start_time}秒")
            print(f"训练次数：{total_train_step}， Loss：{loss.item()}")
            writer.add_scalar(tag='train_loss', scalar_value=loss.item(), global_step=total_train_step)

    # 打印每个epoch的平均loss
    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"第 {i+1} 轮训练，平均 Loss：{avg_train_loss:.4f}")

    # 测试阶段
    model.eval()
    total_test_loss= 0.0
    total_accuracy = 0.0
    avg_test_loss = 0.0

    with torch.no_grad(): # 测试时候不需要计算梯度
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = model(imgs)
            loss = loss_fn(outputs, targets)

            total_test_loss += loss.item()

            # 计算准确率
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy += accuracy

    avg_test_loss = total_test_loss / len(test_dataloader)
    overall_accuracy = total_accuracy / test_data_size
    print(f"第 {i+1} 轮测试，平均 Loss：{avg_test_loss:.4f}，准确率：{overall_accuracy:.2%}")

    writer.add_scalar(tag='test_loss', scalar_value=avg_test_loss, global_step=total_test_step)

    total_test_step += 1

    # 模型保存：每5个epoch保存一次模型
    if (i+1)%5 == 0:
        torch.save(model.state_dict(), f"../checkpoints/CIFRA10/model_epoch_{i+1}.pth")
        print(f"保存第{i+1}轮次的模型")

    # 最佳模型保存
    if overall_accuracy > best_accuracy:
        best_accuracy = overall_accuracy
        torch.save(model.state_dict(), "../checkpoints/CIFRA10/best_model.pth")
        print(f"保存最佳模型！准确率：{best_accuracy:.2%}")

    print()

writer.close()

print(f"🎉 训练完成！最佳准确率：{best_accuracy:.2%}")
print(f"模型已保存到 ../checkpoints/ 目录")

-------第1轮训练开始-------
训练100个batch耗时：1.0568673610687256秒
训练次数：100， Loss：2.2916347980499268
训练100个batch耗时：1.9452755451202393秒
训练次数：200， Loss：2.2843587398529053
训练100个batch耗时：3.2109334468841553秒
训练次数：300， Loss：2.265637159347534
训练100个batch耗时：4.203899145126343秒
训练次数：400， Loss：2.193816900253296
训练100个batch耗时：5.0983195304870605秒
训练次数：500， Loss：2.141592264175415
训练100个batch耗时：6.046863079071045秒
训练次数：600， Loss：2.0545973777770996
训练100个batch耗时：6.990116357803345秒
训练次数：700， Loss：2.003502607345581
第 1 轮训练，平均 Loss：2.1802
第 1 轮测试，平均 Loss：2.0216，准确率：26.57%
保存最佳模型！准确率：26.57%

-------第2轮训练开始-------
训练100个batch耗时：8.865328788757324秒
训练次数：800， Loss：1.8876079320907593
训练100个batch耗时：9.706599950790405秒
训练次数：900， Loss：1.850776195526123
训练100个batch耗时：10.521820783615112秒
训练次数：1000， Loss：1.921422004699707
训练100个batch耗时：11.427802801132202秒
训练次数：1100， Loss：1.9691129922866821
训练100个batch耗时：13.295053958892822秒
训练次数：1200， Loss：1.6913763284683228
训练100个batch耗时：15.294424772262573秒
训练次数：1300， Loss：1.6686904430389404
训练1

In [41]:
# 定义训练的设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 创建网络模型
model = Model_CIFAR()
model.to(device)

# 创建损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device)

# 定义优化器
learning_rate = 2e-2
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)

# 设置训练网络参数
epoch = 20 # 训练轮次
total_train_step = 0 # 记录训练次数
total_test_step = 0 # 记录测试次数
best_accuracy = 0.0 # 记录最佳准确率

# 添加tensorboard
writer = SummaryWriter('../logs/logs_train')

start_time = time.time()
for i in range(epoch):
    print(f"-------第{i+1}轮训练开始-------")

    # 训练开始
    model.train()
    total_train_loss = 0.0
    avg_train_loss = 0.0
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        output = model(imgs)
        loss = loss_fn(output, targets)
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1
        total_train_loss += loss.item()

        # 每100个batch打印一次
        if total_train_step % 100 == 0:
            end_time = time.time()
            print(f"训练100个batch耗时：{end_time - start_time}秒")
            print(f"训练次数：{total_train_step}， Loss：{loss.item()}")
            writer.add_scalar(tag='train_loss', scalar_value=loss.item(), global_step=total_train_step)

    # 打印每个epoch的平均loss
    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"第 {i+1} 轮训练，平均 Loss：{avg_train_loss:.4f}")

    # 测试阶段
    model.eval()
    total_test_loss= 0.0
    total_accuracy = 0.0
    avg_test_loss = 0.0

    with torch.no_grad(): # 测试时候不需要计算梯度
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = model(imgs)
            loss = loss_fn(outputs, targets)

            total_test_loss += loss.item()

            # 计算准确率
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy += accuracy

    avg_test_loss = total_test_loss / len(test_dataloader)
    overall_accuracy = total_accuracy / test_data_size
    print(f"第 {i+1} 轮测试，平均 Loss：{avg_test_loss:.4f}，准确率：{overall_accuracy:.2%}")

    writer.add_scalar(tag='test_loss', scalar_value=avg_test_loss, global_step=total_test_step)

    total_test_step += 1

    # 模型保存：每5个epoch保存一次模型
    if (i+1)%5 == 0:
        torch.save(model.state_dict(), f"../checkpoints/CIFRA10/model_epoch_{i+1}.pth")
        print(f"保存第{i+1}轮次的模型")

    # 最佳模型保存
    if overall_accuracy > best_accuracy:
        best_accuracy = overall_accuracy
        torch.save(model.state_dict(), "../checkpoints/CIFRA10/best_model.pth")
        print(f"保存最佳模型！准确率：{best_accuracy:.2%}")

    print()

writer.close()

print(f"🎉 训练完成！最佳准确率：{best_accuracy:.2%}")
print(f"模型已保存到 ../checkpoints/ 目录")

-------第1轮训练开始-------
训练100个batch耗时：0.3448209762573242秒
训练次数：100， Loss：2.2559256553649902
训练100个batch耗时：0.6811919212341309秒
训练次数：200， Loss：2.1533145904541016
训练100个batch耗时：1.00048828125秒
训练次数：300， Loss：2.041503429412842
训练100个batch耗时：1.3154971599578857秒
训练次数：400， Loss：1.9673103094100952
训练100个batch耗时：1.6318204402923584秒
训练次数：500， Loss：1.8660335540771484
训练100个batch耗时：1.9530961513519287秒
训练次数：600， Loss：1.89186429977417
训练100个batch耗时：2.2703945636749268秒
训练次数：700， Loss：1.8362712860107422
第 1 轮训练，平均 Loss：1.9974
第 1 轮测试，平均 Loss：1.9683，准确率：29.59%
保存最佳模型！准确率：29.59%

-------第2轮训练开始-------
训练100个batch耗时：2.979355812072754秒
训练次数：800， Loss：1.6776424646377563
训练100个batch耗时：3.2981481552124023秒
训练次数：900， Loss：1.5908658504486084
训练100个batch耗时：3.615436315536499秒
训练次数：1000， Loss：1.6286003589630127
训练100个batch耗时：3.933281183242798秒
训练次数：1100， Loss：1.874021291732788
训练100个batch耗时：4.250903129577637秒
训练次数：1200， Loss：1.6056119203567505
训练100个batch耗时：4.575193643569946秒
训练次数：1300， Loss：1.4091897010803223
训练100个

# 4 模型测试

第一步：定义测试图片

In [42]:
img_path = '../images/dog.png'
image = Image.open(img_path)
print(image)

<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=462x314 at 0x726F580DAB30>


第二步：修改图片格式

In [43]:
# 将PNG转换为RGB
image = image.convert('RGB')

transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((32, 32)),
    torchvision.transforms.ToTensor()
])

image = transform(image)

print(image.shape)

torch.Size([3, 32, 32])


In [48]:
image = torch.reshape(input=image, shape=(1, 3, 32, 32))
print(image.shape)

torch.Size([1, 3, 32, 32])


第三步：加载模型权重

In [45]:
model = Model_CIFAR()

model.load_state_dict(torch.load('../checkpoints/CIFRA10/best_model.pth'))

print(model)

Model_CIFAR(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (classfier): Sequential(
    (0): Linear(in_features=1024, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=10, bias=True)
  )
)


第四步：测试

In [ ]:
model.eval()
with torch.no_grad():
    output = model(image)
print(output)
print(output.argmax(1))
print()

tensor([[ -0.1705,  -6.0084,   6.5984,   4.1745,   0.7554,   8.5085,  -5.7296,
           9.7404, -13.5578,  -9.0026]])
tensor([7])
